In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchmetrics
from typing import Optional
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from nn import *
from preprocess import *
from kfold import *
from train import train_one_epoch
from imblearn.over_sampling import RandomOverSampler
from collections import Counter
#from get import get, prepro
#from factory_model import factory_model, create_models, models_cv
#from cv import CV
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/home/estsoft/miniconda3/envs/MathAI/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv("data/pre.csv")

In [3]:
X = get_X(data)
y = get_y(data)[:,np.newaxis]

print(X.shape,y.shape)

#print(X.isnull().sum())
device = torch.device('cpu')
#print(X.shape,y.shape)

X_trn, X_test, y_trn, y_test = split2(X,y)

#SMOTE 사용하기
sm = RandomOverSampler(random_state=42)
X_trn, y_trn = sm.fit_resample(X_trn, np.round(y_trn))

X_trn, X_test, y_trn, y_test = torch.tensor(X_trn), torch.tensor(X_test),torch.tensor(y_trn), torch.tensor(y_test)
X_trn, X_test, y_trn, y_test = X_trn.to(device), X_test.to(device), y_trn.to(device) , y_test.to(device)
'''
import matplotlib.pyplot as plt
plt.hist(y_trn_re, bins='auto')
plt.title("Histogram of Resampled Target Variable")
plt.show()
'''

(37399, 107) (37399, 1)


'\nimport matplotlib.pyplot as plt\nplt.hist(y_trn_re, bins=\'auto\')\nplt.title("Histogram of Resampled Target Variable")\nplt.show()\n'

In [4]:
print(X_trn.shape, y_trn.shape , X_trn.dtype, y_trn.dtype)

#Loader 올리기
ds = TensorDataset(X_trn, y_trn.squeeze().to(torch.long))
dl = DataLoader(ds, batch_size=128, shuffle=True)


torch.Size([597870, 107]) torch.Size([597870]) torch.float32 torch.float32


In [5]:
model = ANN(X_trn.shape[-1],256).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [6]:
kfold_cross_validate(model,criterion=nn.MSELoss(), device=device, X_trn=X_trn, y_trn = y_trn, n_splits=5 , lr=0.0001, epochs=200)

torch.Size([597870, 107])
torch.Size([597870])


  0%|          | 0/200 [00:00<?, ?it/s]/home/estsoft/miniconda3/envs/MathAI/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([256])) that is different to the input size (torch.Size([256, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/estsoft/miniconda3/envs/MathAI/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([88])) that is different to the input size (torch.Size([88, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/home/estsoft/miniconda3/envs/MathAI/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([119574])) that is different to the input size (torch.Size([119574, 1])). Th

RuntimeError: [enforce fail at alloc_cpu.cpp:83] err == 0. DefaultCPUAllocator: can't allocate memory: you tried to allocate 57191765904 bytes. Error code 12 (Cannot allocate memory)